In [ ]:
# Importing the necessary module for SAP HANA data handling
from hana_ml import dataframe
import pandas as pd

# Establishing connection to the SAP HANA database
try:
    conn = dataframe.ConnectionContext(
        address="******",  # IP address of the SAP HANA server
        port="30015",            # Port number for the connection
        user="SYSTEM",           # Username for authentication
        password="*****",     # Password for authentication
        current_schema="GVFPPL_PRODUCTIVE_LIVE_03"  # Schema to be used for the connection
    )
    print("Connection successful")
    
    # Executing SQL query to retrieve data from the specified table
    df = conn.sql("""
        SELECT
    T0."DocNum",
    T0."DocDate",
    T0."CardCode",
    T0."CardName",
    T0."DocTotal",
    T1."ItemCode"
FROM
    GVFPPL_PRODUCTIVE_LIVE_03.OINV T0
INNER JOIN
    GVFPPL_PRODUCTIVE_LIVE_03.INV1 T1 ON T0."DocEntry" = T1."DocEntry"
WHERE
    YEAR(T0."DocDate") >= 2018 AND YEAR(T0."DocDate") <=2023
ORDER BY
    T0."DocDate" DESC;

    """)
    
    # Fetching data
    data = df.collect()
    
    if data.empty:
        print("No data found.")
    else:
        # Specify the path where you want to save the Excel file
        output_path = r"C:\Users\admin\Desktop\output.xlsx"
        
        # Define the maximum number of rows per sheet
        max_rows_per_sheet = 1048500  
        
        # Exporting data to different sheets in Excel
        with pd.ExcelWriter(output_path) as writer:
            sheet_num = 1
            for i in range(0, len(data), max_rows_per_sheet):
                data_subset = data[i:i+max_rows_per_sheet]
                sheet_name = f'Sheet{sheet_num}'
                data_subset.to_excel(writer, sheet_name=sheet_name, index=False)
                sheet_num += 1
        
        print(f"Data exported to {output_path}")
       
except dataframe.dbapi.Error as e:
    print("Data retrieval failed:", e)
except Exception as ex:
    print("An unexpected error occurred:", ex)
